# Use tool calling with LLMs

Enable LLMs to call functions and tools, then execute the results automatically.

## Problem

You want an LLM to decide which functions to call based on user queries—for agents, chatbots, or automated workflows.

| Use case | Tools needed |
|----------|--------------|
| Data assistant | `get_data`, `run_query` |
| Customer support | `lookup_order`, `check_status` |
| Research agent | `search_web`, `fetch_article` |

## Solution

**What's in this recipe:**

- Define tools as Python functions
- Let LLMs decide which tool to call
- Automatically execute tool calls with `invoke_tools`

You define tools with JSON schemas, pass them to the LLM, and use `invoke_tools` to execute the function calls.

### Setup

In [ ]:
%pip install -qU pixeltable openai

In [2]:
import os
import getpass

if 'OPENAI_API_KEY' not in os.environ:
    os.environ['OPENAI_API_KEY'] = getpass.getpass('OpenAI API Key: ')

In [3]:
import pixeltable as pxt
from pixeltable.functions import openai

In [4]:
# Create a fresh directory
pxt.drop_dir('tools_demo', force=True)
pxt.create_dir('tools_demo')

Connected to Pixeltable database at: postgresql+psycopg://postgres:@/pixeltable?host=/Users/pjlb/.pixeltable/pgdata
Created directory 'tools_demo'.


### Define tools as UDFs

In [5]:
# Define tool functions as Pixeltable UDFs
@pxt.udf
def get_weather(city: str) -> str:
    """Get the current weather for a city."""
    # In production, call a real weather API
    weather_data = {
        'new york': 'Sunny, 72°F',
        'london': 'Cloudy, 58°F',
        'tokyo': 'Rainy, 65°F',
        'paris': 'Partly cloudy, 68°F',
    }
    return weather_data.get(city.lower(), f'Weather data not available for {city}')

@pxt.udf
def get_stock_price(symbol: str) -> str:
    """Get the current stock price for a symbol."""
    # In production, call a real stock API
    prices = {
        'AAPL': '$178.50',
        'GOOGL': '$141.25',
        'MSFT': '$378.90',
        'AMZN': '$185.30',
    }
    return prices.get(symbol.upper(), f'Price not available for {symbol}')

In [6]:
# Create a Tools object with our functions
tools = pxt.tools(get_weather, get_stock_price)

### Create tool-calling pipeline

In [7]:
# Create table for queries
queries = pxt.create_table(
    'tools_demo.queries',
    {'query': pxt.String}
)

Created table 'queries'.


In [8]:
# Add LLM call with tools
queries.add_computed_column(
    response=openai.chat_completions(
        messages=[{'role': 'user', 'content': queries.query}],
        model='gpt-4o-mini',
        tools=tools  # Pass tools to the LLM
    )
)

Added 0 column values with 0 errors.


No rows affected.

In [9]:
# Automatically execute tool calls and get results
queries.add_computed_column(
    tool_results=openai.invoke_tools(tools, queries.response)
)

Added 0 column values with 0 errors.


No rows affected.

### Run tool-enabled queries

In [10]:
# Insert queries that require tool calls
sample_queries = [
    {'query': "What's the weather in Tokyo?"},
    {'query': "What's the stock price of Apple?"},
    {'query': "What's the weather in Paris and the price of Microsoft stock?"},
]

queries.insert(sample_queries)

Inserting rows into `queries`: 3 rows [00:00, 330.87 rows/s]
Inserted 3 rows with 0 errors.


3 rows inserted, 9 values computed.

In [11]:
# View results
queries.select(queries.query, queries.tool_results).collect()

query,tool_results
What's the weather in Tokyo?,"{""get_weather"": [""Rainy, 65\u00b0F""], ""get_stock_price"": null}"
What's the stock price of Apple?,"{""get_weather"": null, ""get_stock_price"": [""\$178.50""]}"
What's the weather in Paris and the price of Microsoft stock?,"{""get_weather"": [""Partly cloudy, 68\u00b0F""], ""get_stock_price"": [""\$378.90""]}"


## Explanation

**Tool calling flow:**

```
Query → LLM decides tool → invoke_tools executes → Results
```

**Key components:**

| Component | Purpose |
|-----------|---------|
| `@pxt.udf` | Define tool functions |
| `pxt.tools()` | Bundle functions into Tools object |
| `tools=` parameter | Pass tools to LLM |
| `invoke_tools()` | Execute tool calls from LLM response |

**Supported providers:**

| Provider | Function |
|----------|----------|
| OpenAI | `openai.invoke_tools()` |
| Anthropic | `anthropic.invoke_tools()` |
| Groq | `groq.invoke_tools()` |
| Gemini | `gemini.invoke_tools()` |
| Bedrock | `bedrock.invoke_tools()` |

## See also

- [Build a RAG pipeline](https://docs.pixeltable.com/howto/cookbooks/agents/pattern-rag-pipeline) - Retrieval-augmented generation
- [Run local LLMs](https://docs.pixeltable.com/howto/providers/working-with-ollama) - Local model inference